In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd
import random

In [2]:
#pull single page to test and get max page number
url = 'https://www.clinicaltrialsregister.eu/ctr-search/search?query=&resultsstatus=trials-with-results&page1'
response = get(url, verify = False)
html = response.content

#what does our parsed html look like?
soup = BeautifulSoup(html, "html.parser")

C:\Users\ndevito\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
#gets max page number
number_of_pages = soup.find('div', {'class': 'margin-bottom: 6px;'})
max_page_link = str(number_of_pages.find_all('a')[-1])
max_page = re.findall(r'\d+', max_page_link)[0]
print(max_page)

574


In [4]:
#setting up variables for page URLs
euctr_base_url = 'https://www.clinicaltrialsregister.eu'
euctr_results_search_page = '/ctr-search/search?query=&resultsstatus=trials-with-results&page='

In [11]:
#variables needed for first scrape
pages = [str(i) for i in range(1,int(max_page)+1)]
trial_ids = []
results_urls = []
start_time = time()
requests = 0
print(pages[-1])

574


In [12]:
#this crawls every trial that comes up with a search result of trials with results on the EUCTR

for page in pages:
    
    #make this request
    response = get(euctr_base_url + euctr_results_search_page + page, verify = False)
    
    #pause to look like a human
    #sleep(random.uniform(0,0.2)) #not needed at the moment for this
    
    #mointor the requests to ensure everything is working
    requests += 1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    
    # Throw a warning for a non-200 status code
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #Break the looop if we exceed the number of requests which will need to change when i do full scrape
    if requests > int(max_page):
        warn('Number of requests was greater than expected.')  
        break 
    
    #Parse the requests
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #select all the trial tables
    trial_tables = page_html.find_all('table', {'class': 'result'})
    
    #get the trial id and the trial url for each thing
    for trial_table in trial_tables:
        trial_id = trial_table.input.get('value')
        trial_ids.append(trial_id)
        url = euctr_base_url + trial_table.find_all('a')[-1].get('href')
        results_urls.append(url)

Request: 574; Frequency: 3.640387518614317 requests/s


C:\Users\ndevito\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [23]:
results_trial_id = []
global_end_of_trial_date = []
first_publication_date = []
current_publication_date = []
results_version = []
results_type = []
start_time_2 = time()
requests_2 = 0

def tds_strip(td_table, td):
    return td_table[td].div.get_text().strip()

In [35]:
#this takes the urls from the above scrape, and crawls them for results information

for result_url in results_urls: 
#if scraper fails, see where it failed below with results_df.tail() and add [lastindex:] to the end of results_urls
    
    #make this request
    response = get(result_url, verify = False)
    
    #pause to look like a human
    sleep(random.uniform(0,0.5)) #probably not needed but keeping in for now
    
    #mointor the requests to ensure everything is working
    requests_2 += 1
    elapsed_time = time() - start_time_2
    print('Request: {}; Frequency: {} requests/s'.format(requests_2, requests_2/elapsed_time))
    clear_output(wait = True)
    
    # Throw a warning for a non-200 status code
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests_2, response.status_code))

    #Break the looop if we exceed the number of requests which will need to change when i do full scrape
    if requests_2 > len(results_urls):
        warn('Number of requests was greater than expected.')  
        break 
    
    #Parse the requests
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #select all the results tables
    leg_text = page_html.find('div', id = 'synopsisLegislationNote')
    trial_tables = page_html.find_all('table')[4]
    td_value = trial_tables.find_all('td', class_ = 'valueColumn')
    td_label = trial_tables.find_all('td', class_ = 'labelColumn') 
    
    if td_label[-1].div.get_text().strip() == 'Summary report(s)' and leg_text is not None:
        trial_id = trial_tables.find_all('a')[0].get_text()
        results_trial_id.append(trial_id)
        global_end_date = tds_strip(td_value,3)
        global_end_of_trial_date.append(global_end_date)
        first_pub = tds_strip(td_value,11)
        first_publication_date.append(first_pub)
        current_pub = tds_strip(td_value,10)
        current_publication_date.append(current_pub)
        version = td_value[9].get_text().strip()
        results_version.append(version)
        results_type.append("Document")
    
    elif td_label[-1].div.get_text().strip() == 'Summary report(s)' and leg_text is None:
        trial_id = trial_tables.find_all('a')[0].get_text()
        results_trial_id.append(trial_id)
        global_end_date = tds_strip(td_value,3)
        global_end_of_trial_date.append(global_end_date)
        first_pub = tds_strip(td_value,7)
        first_publication_date.append(first_pub)
        current_pub = tds_strip(td_value,6)
        current_publication_date.append(current_pub)
        version = td_value[5].get_text().strip()
        results_version.append(version)
        results_type.append("Mixed")
        
    else:
        trial_id = trial_tables.find_all('a')[0].get_text()
        results_trial_id.append(trial_id)
        global_end_date = tds_strip(td_value,3)
        global_end_of_trial_date.append(global_end_date)
        first_pub = tds_strip(td_value,7)
        first_publication_date.append(first_pub)
        current_pub = tds_strip(td_value,6)
        current_publication_date.append(current_pub)
        version = td_value[5].get_text().strip()
        results_version.append(version)
        results_type.append("Tabular")

C:\Users\ndevito\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Request: 11476; Frequency: 0.649694465047816 requests/s


In [59]:
results_df = pd.DataFrame({'trial_id': results_trial_id,
                       'global_trial_end_date': global_end_of_trial_date,
                       'first_pub_date': first_publication_date,
                       'current_pub_date': current_publication_date,
                       'version': results_version,
                       'results_type': results_type
                          })

results_df.tail()

,trial_id,global_trial_end_date,first_pub_date,current_pub_date,version,results_type
11471,2017-000952-24,28 Feb 2018,07 Feb 2019,07 Feb 2019,v1(current),Tabular
11472,2010-024132-41,,16 Mar 2017,16 Mar 2017,v1(current),Tabular
11473,2010-021978-11,13 Apr 2016,29 Apr 2017,26 Aug 2017,v3(current),Tabular
11474,2014-001498-15,14 Mar 2016,24 Nov 2016,24 Nov 2016,v1(current),Tabular
11475,2010-019630-28,13 Nov 2015,17 Feb 2017,17 Feb 2017,v1(current),Tabular


In [60]:
#some data cleaning

for index, row in results_df.iterrows():
    if row.first_pub_date == 'No':
        results_df.at[index, 'first_pub_date'] = None
        results_df.at[index, 'current_pub_date'] = None
        results_df.at[index, 'version'] = None
        results_df.at[index, 'results_type'] = 'None Available'

In [61]:
#aates to datetime

results_df['global_trial_end_date'] = pd.to_datetime(results_df['global_trial_end_date'])
results_df['first_pub_date'] = pd.to_datetime(results_df['first_pub_date'])
results_df['current_pub_date'] = pd.to_datetime(results_df['current_pub_date'])

In [62]:
results_df.dtypes

trial_id                         object
global_trial_end_date    datetime64[ns]
first_pub_date           datetime64[ns]
current_pub_date         datetime64[ns]
version                          object
results_type                     object
dtype: object

In [64]:
results_df.tail()

,trial_id,global_trial_end_date,first_pub_date,current_pub_date,version,results_type
11471,2017-000952-24,2018-02-28,2019-02-07,2019-02-07,v1(current),Tabular
11472,2010-024132-41,NaT,2017-03-16,2017-03-16,v1(current),Tabular
11473,2010-021978-11,2016-04-13,2017-04-29,2017-08-26,v3(current),Tabular
11474,2014-001498-15,2016-03-14,2016-11-24,2016-11-24,v1(current),Tabular
11475,2010-019630-28,2015-11-13,2017-02-17,2017-02-17,v1(current),Tabular


In [58]:
#make a csv
results_df.to_csv('euctr_results_scrape.csv')